In [ ]:
import pandas as pd

def process_excel(filename):
    """
    Reads an Excel file, extracts organization names and creates a combined table.

    Args:
        filename: The name of the Excel file.

    Returns:
        A pandas DataFrame containing the combined table, or None if an error occurs.
    """
    try:
        df = pd.read_excel(filename)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {e}")
        return None

    organization_names = []
    data_rows = []
    current_org_name = None

    for index, row in df.iterrows():
        if row.iloc[0] == "Organization Name":  # Check first cell in row
            current_org_name = row.iloc[1] # Get the name right next to it
        elif current_org_name is not None and row.iloc[0] == "Store Code":
            # Start of a data table
            header = row.tolist()
            header.insert(0,"Organization Name") #insert org name
            data_rows.append(header)
            for i in range(index+1, len(df)):
                if pd.isna(df.iloc[i,0]) or df.iloc[i,0] == "": #check for empty rows
                    continue
                elif df.iloc[i,0] == "Amounts Sum:": #check for end of table
                    break
                else:
                    data_row = df.iloc[i].tolist()
                    data_row.insert(0,current_org_name) #insert org name
                    data_rows.append(data_row)
            current_org_name = None # reset current org name

    if data_rows:
      final_df = pd.DataFrame(data_rows[1:], columns=data_rows[0])
      return final_df
    else:
      print("No data tables found in the Excel file.")
      return None



# Example usage:
filename = "report - Copy.xlsx"  # Replace with your file name
final_table = process_excel(filename)

if final_table is not None:
    print(final_table)
    # Save to a new Excel file (optional):
    final_table.to_excel("combined_table.xlsx", index=False)
    print("Combined table saved to combined_table.xlsx")